# Teacher scheduling with PuLP

This is an example notebook on how to solve the teacher scheduling problem with Linear Programming in Python using PuLP. Different algorithms to follow.

## Imports

In [1]:
import pulp

## Create the problem

In [216]:
# Create the LP problem
teacher_schedule = pulp.LpProblem("Teacher_Schedule", pulp.LpMinimize)

## Define decision variables

In [217]:
# Define the decision variables
# For simplicity, let's assume you have 3 teachers (T1, T2, T3), 2 classes (C1, C2), and 2 time slots (M1, M2).
teachers = ['T1', 'T2', 'T3']
classes = ['C1', 'C2']
time_slots = ['M1', 'M2', 'M3', 'M4']

In [218]:
# pulp.makeDict()

In [219]:
# Create a binary variable for each teacher, class, and time slot combination
# x = pulp.LpVariable.dicts("Assignment", ((teacher, _class, time) for teacher in teachers for _class in classes for time in time_slots), cat='Binary')
x = pulp.LpVariable.dicts(
    "Assignment",
    ((teacher, _class, time) for teacher in teachers for _class in classes for time in time_slots),
    # lowBound=0,
    # upBound=1,
    cat=pulp.LpBinary)

In [220]:
x[('T1', 'C1', 'M1')] == 0

1*Assignment_('T1',_'C1',_'M1') + 0 = 0

## Define constraints

In [226]:
# Example constraint: Teacher availability
# Ensure that each teacher is scheduled only during their available time slots

teacher_availability = {
    'T1': {'M1', 'M2', 'M3', 'M4'},
    'T2': {'M1', 'M2', 'M3', 'M4'},
    'T3': {'M1', 'M2', 'M3', 'M4'},
}

for teacher in teachers:
    for time in time_slots:
        if time in teacher_availability[teacher]:
            for _class in classes:
                # If a teacher is available at a specific time, the associated variable must be set to 1 for all classes
                teacher_schedule += x[(teacher, _class, time)] == 1
            # Example constraint: Teacher exclusivity for time slots

# Ensure each class for a teacher can only be taught in one time slot
for teacher in teachers:
    for _class in classes:
        teacher_schedule += pulp.lpSum(x[(teacher, _class, time)] for time in time_slots) <= 1

# Ensure that each teacher is assigned to only one class during a specific time slot
for teacher in teachers:
    for time in time_slots:
        teacher_schedule += pulp.lpSum(x[(teacher, _class, time)] for _class in classes) <= 1

In [222]:
teacher_schedule

Teacher_Schedule:
MINIMIZE
None
SUBJECT TO
_C1: Assignment_('T1',_'C1',_'M1') = 1

_C2: Assignment_('T1',_'C2',_'M1') = 1

_C3: Assignment_('T1',_'C1',_'M2') = 1

_C4: Assignment_('T1',_'C2',_'M2') = 1

_C5: Assignment_('T1',_'C1',_'M3') = 1

_C6: Assignment_('T1',_'C2',_'M3') = 1

_C7: Assignment_('T1',_'C1',_'M4') = 1

_C8: Assignment_('T1',_'C2',_'M4') = 1

_C9: Assignment_('T2',_'C1',_'M1') = 1

_C10: Assignment_('T2',_'C2',_'M1') = 1

_C11: Assignment_('T2',_'C1',_'M2') = 1

_C12: Assignment_('T2',_'C2',_'M2') = 1

_C13: Assignment_('T2',_'C1',_'M3') = 1

_C14: Assignment_('T2',_'C2',_'M3') = 1

_C15: Assignment_('T2',_'C1',_'M4') = 1

_C16: Assignment_('T2',_'C2',_'M4') = 1

_C17: Assignment_('T3',_'C1',_'M1') = 1

_C18: Assignment_('T3',_'C2',_'M1') = 1

_C19: Assignment_('T3',_'C1',_'M2') = 1

_C20: Assignment_('T3',_'C2',_'M2') = 1

_C21: Assignment_('T3',_'C1',_'M3') = 1

_C22: Assignment_('T3',_'C2',_'M3') = 1

_C23: Assignment_('T3',_'C1',_'M4') = 1

_C24: Assignment_('T3',

## Define Objective Function

In [223]:
# Example objective function: Minimize teacher idle time
# Define the objective: Minimize teacher idle time while considering availability
teacher_idle_time = pulp.lpSum((1 - x[(teacher, _class, time)]) for teacher in teachers for _class in classes for time in time_slots)
# Add the objective function to the LP problem
teacher_schedule += teacher_idle_time


In [224]:
teacher_schedule

Teacher_Schedule:
MINIMIZE
-1*Assignment_('T1',_'C1',_'M1') + -1*Assignment_('T1',_'C1',_'M2') + -1*Assignment_('T1',_'C1',_'M3') + -1*Assignment_('T1',_'C1',_'M4') + -1*Assignment_('T1',_'C2',_'M1') + -1*Assignment_('T1',_'C2',_'M2') + -1*Assignment_('T1',_'C2',_'M3') + -1*Assignment_('T1',_'C2',_'M4') + -1*Assignment_('T2',_'C1',_'M1') + -1*Assignment_('T2',_'C1',_'M2') + -1*Assignment_('T2',_'C1',_'M3') + -1*Assignment_('T2',_'C1',_'M4') + -1*Assignment_('T2',_'C2',_'M1') + -1*Assignment_('T2',_'C2',_'M2') + -1*Assignment_('T2',_'C2',_'M3') + -1*Assignment_('T2',_'C2',_'M4') + -1*Assignment_('T3',_'C1',_'M1') + -1*Assignment_('T3',_'C1',_'M2') + -1*Assignment_('T3',_'C1',_'M3') + -1*Assignment_('T3',_'C1',_'M4') + -1*Assignment_('T3',_'C2',_'M1') + -1*Assignment_('T3',_'C2',_'M2') + -1*Assignment_('T3',_'C2',_'M3') + -1*Assignment_('T3',_'C2',_'M4') + 24
SUBJECT TO
_C1: Assignment_('T1',_'C1',_'M1') = 1

_C2: Assignment_('T1',_'C2',_'M1') = 1

_C3: Assignment_('T1',_'C1',_'M2') = 1


In [225]:
# Solve the optimization model
# Currently having all 3 constraints displays problem infeasible. This is still a work in progress
teacher_schedule.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sashlinreddy/mambaforge/envs/optimization/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/qk/w6lvkswx5vnfvj7zlfl28zqm0000gn/T/6ff92f45c9fb4e6e9f4c14f2f168e7c5-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/qk/w6lvkswx5vnfvj7zlfl28zqm0000gn/T/6ff92f45c9fb4e6e9f4c14f2f168e7c5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 47 COLUMNS
At line 192 RHS
At line 235 BOUNDS
At line 260 ENDATA
Problem MODEL has 42 rows, 24 columns and 72 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



-1

In [205]:
# Check the status of the solution
if pulp.LpStatus[teacher_schedule.status] == "Optimal":
    print("Optimal solution found")
    # Extract and use the results
    for teacher in teachers:
        for _class in classes:
            for time in time_slots:
                if pulp.value(x[(teacher, _class, time)]) == 1:
                    print(f"Teacher {teacher} is assigned to Class {_class} at Time Slot {time}")
else:
    print("No optimal solution found. Check your constraints and objectives.")

Optimal solution found
Teacher T1 is assigned to Class C1 at Time Slot M1
Teacher T1 is assigned to Class C1 at Time Slot M2
Teacher T1 is assigned to Class C1 at Time Slot M3
Teacher T1 is assigned to Class C1 at Time Slot M4
Teacher T1 is assigned to Class C2 at Time Slot M1
Teacher T1 is assigned to Class C2 at Time Slot M2
Teacher T1 is assigned to Class C2 at Time Slot M3
Teacher T1 is assigned to Class C2 at Time Slot M4
Teacher T2 is assigned to Class C1 at Time Slot M1
Teacher T2 is assigned to Class C1 at Time Slot M2
Teacher T2 is assigned to Class C1 at Time Slot M3
Teacher T2 is assigned to Class C1 at Time Slot M4
Teacher T2 is assigned to Class C2 at Time Slot M1
Teacher T2 is assigned to Class C2 at Time Slot M2
Teacher T2 is assigned to Class C2 at Time Slot M3
Teacher T2 is assigned to Class C2 at Time Slot M4
Teacher T3 is assigned to Class C1 at Time Slot M1
Teacher T3 is assigned to Class C1 at Time Slot M2
Teacher T3 is assigned to Class C1 at Time Slot M3
Teacher 